# Análise exploratória dos dados 

In [ ]:
import os
from pathlib import Path

# Caminho absoluto da raiz do projeto
project_root = Path.cwd().parent  # se o notebook está em /notebooks
os.chdir(project_root)

print("Diretório atual:", Path.cwd())

In [ ]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns
from src.modules.db_reader import DbReader
f1_db = DbReader()

# Olhada geral nos dados de qualificação:

In [ ]:
df_quali = f1_db.run_query_file("data/db_queries/qualify_report.sql").drop_duplicates()

In [ ]:
df_quali

In [ ]:
df_poles = df_quali[df_quali["position"] == 1].copy()

In [ ]:
df_poles

In [ ]:
df_count_pole_por_piloto = df_poles.groupby("driver_surname").size().reset_index(name="pole_count")

In [ ]:
df_count_pole_por_piloto

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

sns.barplot(
    data=df_count_pole_por_piloto.sort_values("pole_count", ascending=False).head(20),
    y="pole_count",
    x="driver_surname",
    palette="viridis",
    ax=ax
)

# Ajusta rótulos no eixo X
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Adiciona valores acima das barras
for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', padding=3)

# Ajusta margem superior para os rótulos não ficarem cortados
plt.tight_layout()
plt.show()


Vendo o gráfico acima, parece que o dataset não é 100% perfeito, o Schumacher teria mais poles do que o indicado, e o Hamilton por exemplo não tem 107, e sim 104.

## Vou tentar analisar a integralidade dos dados, e o quão errado pode estar:

### Primeiro vou ver a quantidade de dados por ano:

In [ ]:
df_quali

In [ ]:
df_eventos_quali_por_ano = df_quali[["year", "race_name"]].drop_duplicates().sort_values("year")

In [ ]:
df_eventos_quali_por_ano

In [ ]:
fing, ax = plt.subplots(figsize=(12, 8))

sns.barplot(
    data=df_eventos_quali_por_ano.groupby("year").size().reset_index(name="count"),
    x="year",
    y="count",
    palette="viridis",
    ax=ax
)

# Ajusta rótulos no eixo X
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Adiciona valores acima das barras
for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', padding=3)

# Ajusta margem superior para os rótulos não ficarem cortados
plt.tight_layout()
plt.show()


Olhando o gráfico acima e dando uma pesquisada, parece que os dados estão completos a partir de 2003.

### Tentando entender porque o Hamilton tem pole a mais do que deveria:

In [ ]:
df_quali_ham = df_quali[df_quali["driver_surname"] == "Hamilton"].copy()

In [ ]:
df_quali_ham

In [ ]:
df_poles_ham = df_quali_ham[df_quali_ham["position"] == 1].copy()

In [ ]:
df_poles_ham

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

sns.barplot(
    data=df_poles_ham.groupby("year").size().reset_index(name="pole_count"),
    x="year",
    y="pole_count",
    palette="viridis",
    ax=ax
)

# Ajusta rótulos no eixo X
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Adiciona valores acima das barras
for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', padding=3)

# Ajusta margem superior para os rótulos não ficarem cortados
plt.tight_layout()
plt.show()

Por exemplo, aqui fala que o Hamilton tem 5 poles em 2007, mas em tese deveria ser 6. E o que parece que tá faltando é a pole dele na Hungria.

In [ ]:
df_quali[(df_quali["year"] == 2007) & (df_quali["race_name"] == "Hungarian Grand Prix")]

Analisando os dados mais a fundo e pesquisando sobre o GP da Hungria: o Hamilton oficialmente fez a pole, porém o que aconteceu foi que o na verdade quem fez o melhor tempo foi o Alonso (conforme os dados mostram corretamente), mas o Alonso foi penalizado e não largou em primeiro na corrida. Então o dataset está correto no sentido de mostrar quem fez os melhores tempos mas não necessariamente vai mostrar quem vai largar em primeiro de fato caso tenha acontecido alguma penalização. Isso potencialmente pode ser corrigido pelo dataset de corridas, que tem a posição de largada de cada piloto.

Ao mesmo tempo, em 2012 ele teria feito 7 poles e não 8, vamos entender:

In [ ]:
df_quali_ham_2012 = df_quali_ham[df_quali_ham["year"] == 2012].copy()

In [ ]:
df_quali_ham_2012

Analisando os dados e dando uma pesquisada a respeito chego em conclusões parecidas com o que vi acima sobre o GP das Hungria de 2007: em 2012 no GP da Espanha o Hamilton fez a volta mais rápida (como os dados mostram), mas na verdade ele largou de último porque o carro dele estava irregular. Então em termos de analisar os tempos, parece que esse dataset vai estar correto para os dados que estão presentes mas não vai respeitar eventuais desclassificações etc sendo que imagino que isso vai ser respeitado no dataset de corridas (o que faz sentido). Em breve, vou dar uma olhada no dataset de corridas pra confirmar. 

# Dados de Corrida: